COS80024

DATA SCIENCE PROJECT 1

PROJECT 4: MOVIE RECOMMENDATION SYSTEM

# Task S3.2 Creating Training and Test Sets

This task aims to generation of training and test data sets.

Code Contributed by - Asif Rana, Student ID - 102866893

In [1]:
# Importing pandas library
import pandas as pd
import numpy as np
import random

In [2]:
# Importing the csv data files 
metadata = pd.read_csv('metadata_integrated.csv',error_bad_lines=False, warn_bad_lines=True)
rating_data = pd.read_csv('ratings_integrated.csv',error_bad_lines=False, warn_bad_lines=True)

In [1]:
# Setting the test sample size
test_sample_size = 5

In [7]:
unique_user_ID = rating_data['userId'].unique()
print(unique_user_ID)
print(len(unique_user_ID))

unique_movie_ID = metadata['movieId'].unique()
print(unique_movie_ID)
print(len(unique_movie_ID))

[  1   7  31  32  36  39  73  88  96 110 111 150 161 165 186 242 254 288
 310 311 325 338 341 386 394 452 457 461 468 485 487 496 511 516 525 590
 596 601 607 619 639 641  19  35  61  99 102 128 175 185 188 213 214 268
 294 303 363 387 420 442 466 471 472 514 518 529 530 547 551 564 575 577
 582 602 603 605 646 665  23  30  70  90 105 119 141 212 233 252 292 306
 318 380 507 509 534 537 584 608 647  21  34  41  49  57  66  77 103 118
 155 177 195 248 276 299 346 351 353 358 390 423 463 522 580 585 597 624
 634 654  38  56  94 130 133 148 198 229 232 262 280 283 320 321 330 372
 373 391 430 441 460 481 497 505 510 521 539 545 587  15  17  22 152 257
 265 309 344 370 434 527 533 592 648 661 137 207 313 326 335 388 404 405
 470 488 559 598  20  48  78 211 236 281 419 532 576  81  95 243 247 285
 290 297 300 308 312 337 362 403 560 562 614 617 618 658 664  27 120 291
 342 407 613   4 202 217 439 458 477 546 574  25  26  87 154 224 361 384
 428 502 586 606 220 360 431 436 520 594 669  59 19

In [8]:
rating_data['movieId'].nunique()

9066

In [9]:
metadata['movieId'].nunique()

9082

### Creating the Cartesian Product for all possible combination of UserId and MovieId

In [10]:
# df containing only the userID column
user_df = pd.DataFrame(unique_user_ID, columns = ['userId'])
# Df containing only the movieID column
movies_df = pd.DataFrame(unique_movie_ID, columns = ['movieId'])
print(user_df.head())
print(user_df.size)

print(movies_df.head())
print(movies_df.size)

# Creating the Cartecian Product of Users and Movies
user_movie_product = (
    user_df.assign(key=1)
    .merge(movies_df.assign(key=1), on="key")
    .drop("key", axis=1)
)
print(user_movie_product.head())
print(user_movie_product.info())

   userId
0       1
1       7
2      31
3      32
4      36
671
   movieId
0        1
1        2
2        3
3        4
4        5
9082
   userId  movieId
0       1        1
1       1        2
2       1        3
3       1        4
4       1        5
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6094022 entries, 0 to 6094021
Data columns (total 2 columns):
userId     int64
movieId    int64
dtypes: int64(2)
memory usage: 139.5 MB
None


In [11]:
# Query for Cross Checking
user_movie_product.loc[(user_movie_product['userId'] == 1) & (user_movie_product['movieId'] == 1029)]

,userId,movieId
830,1,1029


In [12]:
user_movie_product.loc[(user_movie_product['userId'] == 96)  & (user_movie_product['movieId'] == 7587)]

,userId,movieId
77838,96,7587


In [13]:
user_movie_product.head(4)

,userId,movieId
0,1,1
1,1,2
2,1,3
3,1,4


In [14]:
rating_data.head(4)

,userId,movieId,rating,imdbId,tmdbId
0,1,31,2.5,112792,9909.0
1,7,31,3.0,112792,9909.0
2,31,31,4.0,112792,9909.0
3,32,31,4.0,112792,9909.0


In [15]:
rating_data.loc[(rating_data['userId'] == 1) & (rating_data['movieId'] == 1029)]

,userId,movieId,rating,imdbId,tmdbId
42,1,1029,3.0,33563,11360.0


### Merging the dataframes to get the rated and unrated movies for each user

In [16]:
# Merging two dataframes to get the rated and unrated movies
merged_rating_df = pd.merge(user_movie_product, rating_data, how='left', left_on=['userId','movieId'], right_on = ['userId','movieId']).drop(columns = ['imdbId','tmdbId'])

In [17]:
merged_rating_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6094022 entries, 0 to 6094021
Data columns (total 3 columns):
userId     int64
movieId    int64
rating     float64
dtypes: float64(1), int64(2)
memory usage: 186.0 MB


In [18]:
# Cross Checking
merged_rating_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6094022 entries, 0 to 6094021
Data columns (total 3 columns):
userId     int64
movieId    int64
rating     float64
dtypes: float64(1), int64(2)
memory usage: 186.0 MB


In [19]:
merged_rating_df.loc[(merged_rating_df['userId'] == 1) & (merged_rating_df['movieId'] == 1029)]

,userId,movieId,rating
830,1,1029,3.0


In [20]:
merged_rating_df.loc[(merged_rating_df['userId'] == 227) & (merged_rating_df['movieId'] == 162542)]

,userId,movieId,rating
6094018,227,162542,NaN


## Splitting rated and unrated movies for each user

In [21]:
# Dataframe containg only the rated movies by the users
rated_movies = merged_rating_df
rated_movies = rated_movies.dropna()

print(rated_movies.tail())
print(rated_movies.size)
print(rated_movies.info())


bool_series = pd.isnull(merged_rating_df['rating'])
#print(bool_series)  

# Dataframe containg only the un-rated movies by the users
unrated_movies = merged_rating_df[bool_series] 

print(unrated_movies.tail())
print(unrated_movies.size)
print(unrated_movies.info())

         userId  movieId  rating
6086831     227     2385     2.0
6086832     227     2386     5.0
6086833     227     2387     5.0
6086835     227     2389     5.0
6086838     227     2392     4.0
299430
<class 'pandas.core.frame.DataFrame'>
Int64Index: 99810 entries, 30 to 6086838
Data columns (total 3 columns):
userId     99810 non-null int64
movieId    99810 non-null int64
rating     99810 non-null float64
dtypes: float64(1), int64(2)
memory usage: 3.0 MB
None
         userId  movieId  rating
6094017     227   161944     NaN
6094018     227   162542     NaN
6094019     227   162672     NaN
6094020     227   163056     NaN
6094021     227   163949     NaN
17982636
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5994212 entries, 0 to 6094021
Data columns (total 3 columns):
userId     int64
movieId    int64
rating     float64
dtypes: float64(1), int64(2)
memory usage: 182.9 MB
None


In [22]:
rated_movies.loc[(rated_movies['userId'] == 96) & (rated_movies['movieId'] == 7587)]

,userId,movieId,rating
77838,96,7587,4.0


## Creating test dataset for each user

In [23]:
# Initializing dataframe
column_names = ['userId','movieId','rating']
test_df = pd.DataFrame(columns = column_names)
user_test_df = pd.DataFrame(columns = column_names)

# The follwing selects '5' rated movies for each user for the test dataset
for i in range(len(unique_user_ID)):
    temp = rated_movies.loc[rated_movies['userId'] == unique_user_ID[i]]
    temp_movies = temp['movieId'].tolist()
    sample_movies = random.sample(temp_movies, test_sample_size)
    #print("Random item from list is: ", sample_movies)
    for j in range(len(sample_movies)):
        user_test_df = rated_movies.loc[(rated_movies['userId'] == unique_user_ID[i]) & (rated_movies['movieId'] == sample_movies[j])]
        test_df = pd.concat([test_df, user_test_df])

In [24]:
test_df.head(5)

,userId,movieId,rating
1958,1,2455,2.5
927,1,1172,4.0
856,1,1061,3.0
1739,1,2193,2.0
1136,1,1405,1.0


In [25]:
test_df['userId'] = test_df['userId'].astype(str).astype(int)
test_df['movieId'] = test_df['movieId'].astype(str).astype(int)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3355 entries, 1958 to 6086830
Data columns (total 3 columns):
userId     3355 non-null int32
movieId    3355 non-null int32
rating     3355 non-null float64
dtypes: float64(1), int32(2)
memory usage: 78.6 KB


In [26]:
rated_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99810 entries, 30 to 6086838
Data columns (total 3 columns):
userId     99810 non-null int64
movieId    99810 non-null int64
rating     99810 non-null float64
dtypes: float64(1), int64(2)
memory usage: 3.0 MB


## Creating test dataset for each user

In [27]:
# Creating the Train dataset

train_df = pd.merge(rated_movies, test_df, how='left', left_on=['userId','movieId'], right_on = ['userId','movieId'])

bool_series = pd.isnull(train_df['rating_y'])

train_df = train_df[bool_series] 
train_df = train_df.drop(['rating_y'], axis = 1)
train_df = train_df.rename(columns={'rating_x': 'rating'})

In [28]:
train_df.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
3,1,1129,2.0
5,1,1263,2.0
6,1,1287,2.0


In [29]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96455 entries, 0 to 99809
Data columns (total 3 columns):
userId     96455 non-null int64
movieId    96455 non-null int64
rating     96455 non-null float64
dtypes: float64(1), int64(2)
memory usage: 2.9 MB


In [30]:
# Integrating the features of movie metadata to the train and test dataset
train_df = pd.merge(train_df, metadata, how='left', left_on=['movieId'], right_on = ['movieId'])
test_df = pd.merge(test_df, metadata, how='left', left_on=['movieId'], right_on = ['movieId'])
train_df.head()

,userId,movieId,rating,genres,tmdbId,imdb_id,original_language,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,collectionId,collectionName,year
0,1,31,2.5,"Drama,Crime",9909,112792,en,Former Marine Louanne Johnson lands a gig teac...,9.481338,"Hollywood Pictures,Via Rosa Productions,Don Si...",...,English,Released,She broke the rules... and changed their lives.,Dangerous Minds,False,6.4,249.0,-99,unknown,1995
1,1,1029,3.0,"Animation,Family",11360,33563,en,Dumbo is a baby elephant born with oversized e...,14.655879,"RKO Radio Pictures,Walt Disney Productions",...,English,Released,The One...The Only...The FABULOUS...,Dumbo,False,6.8,1206.0,-99,unknown,1941
2,1,1129,2.0,"Science Fiction,Action",1103,82340,en,"In 1997, the island of Manhattan has been wall...",10.124544,"AVCO Embassy Pictures,Goldcrest Films Internat...",...,English,Released,1997. New York City is now a maximum security ...,Escape from New York,False,6.9,720.0,115838,'Escape From ... Collection',1981
3,1,1263,2.0,"Drama,War",11778,77416,en,A group of working-class friends decides to en...,7.729445,"Universal Pictures,EMI Films Ltd.",...,"English,Français,Pусский,Tiếng Việt",Released,One of the most important and powerful films o...,The Deer Hunter,False,7.8,943.0,-99,unknown,1978
4,1,1287,2.0,"Action,Adventure,Drama,History",665,52618,en,Ben-Hur is a 1959 epic film directed by Willia...,14.319743,Metro-Goldwyn-Mayer (MGM),...,English,Released,The entertainment experience of a lifetime.,Ben-Hur,False,7.5,660.0,-99,unknown,1959


In [31]:
test_df.head()

,userId,movieId,rating,genres,tmdbId,imdb_id,original_language,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,collectionId,collectionName,year
0,1,2455,2.5,"Horror,Science Fiction",9426,91064,en,When Seth Brundle makes a huge scientific and ...,13.534589,"SLM Production Group,Brooksfilms",...,English,Released,Be afraid. Be very afraid.,The Fly,False,7.1,1038.0,109609,'The Fly Collection',1986
1,1,1172,4.0,"Drama,Romance",11216,95765,it,"A filmmaker recalls his childhood, when he fel...",14.177005,Rai Tre Radiotelevisione Italiana,...,Italiano,Released,"A celebration of youth, friendship, and the ev...",Cinema Paradiso,False,8.2,834.0,-99,unknown,1988
2,1,1061,3.0,"Crime,Drama,Thriller",819,117665,en,Two gangsters seek revenge on the state jail w...,9.198283,"Propaganda Films,Warner Bros.,Baltimore Pictures",...,English,Released,When friendship runs deeper than blood.,Sleepers,False,7.3,729.0,-99,unknown,1996
3,1,2193,2.0,"Adventure,Drama,Action",847,96446,en,Fearful of a prophecy stating that a girl chil...,9.918312,"Lucasfilm,Metro-Goldwyn-Mayer (MGM),Imagine Fi...",...,English,Released,Adventure doesn't come any bigger than this.,Willow,False,6.9,484.0,-99,unknown,1988
4,1,1405,1.0,"Animation,Comedy",3179,115641,en,"Mike Judge's slacker duo, Beavis and Butt-Head...",8.855685,"Paramount Pictures,Geffen Pictures,MTV Films",...,"Český,English",Released,Coming to a screen bigger than your TV.,Beavis and Butt-Head Do America,False,6.5,169.0,-99,unknown,1996


In [32]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96494 entries, 0 to 96493
Data columns (total 22 columns):
userId                  96494 non-null int64
movieId                 96494 non-null int64
rating                  96494 non-null float64
genres                  96494 non-null object
tmdbId                  96494 non-null int64
imdb_id                 96494 non-null int64
original_language       96494 non-null object
overview                96494 non-null object
popularity              96494 non-null float64
production_companies    96494 non-null object
production_countries    96494 non-null object
runtime                 96494 non-null float64
spoken_languages        96494 non-null object
status                  96494 non-null object
tagline                 96494 non-null object
title                   96494 non-null object
video                   96494 non-null bool
vote_average            96494 non-null float64
vote_count              96494 non-null float64
collectionId      

In [33]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3356 entries, 0 to 3355
Data columns (total 22 columns):
userId                  3356 non-null int32
movieId                 3356 non-null int32
rating                  3356 non-null float64
genres                  3356 non-null object
tmdbId                  3356 non-null int64
imdb_id                 3356 non-null int64
original_language       3356 non-null object
overview                3356 non-null object
popularity              3356 non-null float64
production_companies    3356 non-null object
production_countries    3356 non-null object
runtime                 3356 non-null float64
spoken_languages        3356 non-null object
status                  3356 non-null object
tagline                 3356 non-null object
title                   3356 non-null object
video                   3356 non-null bool
vote_average            3356 non-null float64
vote_count              3356 non-null float64
collectionId            3356 non-null i

In [34]:
#Exporting and train and test dataset for the users.

train_df.to_csv("train_df.csv", index=False)
test_df.to_csv("test_df.csv", index=False)

# Expoting the unrated movies by the user 
unrated_movies.to_csv("unrated_movies.csv", index=False)